In [65]:
# Core & Gmail imports
import base64
import pickle
import os
from email.mime.text import MIMEText

# Google Gmail API
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# LLM + UI
from openai import OpenAI


In [67]:
SCOPES = ["https://www.googleapis.com/auth/gmail.send"]


In [68]:
def send_gmail(to_email, subject, body):
    creds = None

    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                os.path.join(os.getcwd(), "credentials.json"),
                SCOPES
            )
            creds = flow.run_local_server(port=0)

        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    service = build("gmail", "v1", credentials=creds)

    # Send as HTML so spacing looks correct
    message = MIMEText(body.replace("\n", "<br>"), "html")
    message["to"] = to_email
    message["subject"] = subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    service.users().messages().send(
        userId="me", body={"raw": raw}
    ).execute()


In [69]:
openai = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)


In [70]:
SYSTEM_PROMPT = """
You are an intelligent email-writing assistant.

STRICT RULES (must follow exactly):
- Always output in Markdown
- Always include these exact headings:
  ## Subject
  ## Email Body
- Do NOT change the heading text
- Do NOT omit any section
"""


In [71]:
def generate_email(context, key_points, name):
    user_prompt = f"""
Write an email using the following details:

Context:
{context}

Key points:
{key_points}

Sign-off name:
{name}
"""
    response = openai.chat.completions.create(
        model="llama3.2",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content


In [72]:
def parse_email(md_text):
    subject = "No Subject"
    body = md_text.strip()

    if "## Subject" in md_text:
        part = md_text.split("## Subject", 1)[1]
        if "## Email Body" in part:
            subject = part.split("## Email Body", 1)[0].strip()
            body = part.split("## Email Body", 1)[1].strip()

    return subject, body


In [73]:
LAST_GENERATED_EMAIL = None


In [74]:
def preview_email(context, key_points, name):
    global LAST_GENERATED_EMAIL
    LAST_GENERATED_EMAIL = generate_email(context, key_points, name)
    return LAST_GENERATED_EMAIL


In [75]:
def send_previewed_email(to_email):
    global LAST_GENERATED_EMAIL

    if LAST_GENERATED_EMAIL is None:
        raise ValueError("No email generated yet. Please preview first.")

    subject, body = parse_email(LAST_GENERATED_EMAIL)
    send_gmail(to_email, subject, body)

    return LAST_GENERATED_EMAIL


In [64]:
def preview_email(context, key_points, name):
    global LAST_GENERATED_EMAIL
    LAST_GENERATED_EMAIL = generate_email(context, key_points, name)
    return LAST_GENERATED_EMAIL


In [ ]:
preview_email(
    context="Academic lab absence",
    key_points="- Missed lab due to academic commitment\n- Instructor informed earlier\n- Request no mark deduction\n- Will submit assignment next lab",
    name="Sudhanshu Batra"
)


"## Subject\nRequest Clarification on Absence in Recent Lab Session\n\n## Email Body\nDear \\[Instructor's Name],\n\nI am writing to inform you of my absence from the recent lab session, which I understand was missed due to ongoing academic commitments. As per our previous discussions, I had informed you about such schedule constraints earlier and apologize for any inconvenience this might have caused.\n\nAs a result of missing the lab session, I would like to request that no mark deduction be made against me. This will enable me to maintain my overall performance without undue penalty.\n\nFortunately, I am committed to keeping up with the coursework. Next week's lab session should feature an assignment due on the same day as our scheduled presentation; you are welcome to direct me there for more information regarding submission details if needed.\n\n\n\nBest regards,\nNaman Gupta"

In [77]:
send_previewed_email("batra.sudhanshu04@gmail.com")


"## Subject\nRequest Clarification on Absence in Recent Lab Session\n\n## Email Body\nDear \\[Instructor's Name],\n\nI am writing to inform you of my absence from the recent lab session, which I understand was missed due to ongoing academic commitments. As per our previous discussions, I had informed you about such schedule constraints earlier and apologize for any inconvenience this might have caused.\n\nAs a result of missing the lab session, I would like to request that no mark deduction be made against me. This will enable me to maintain my overall performance without undue penalty.\n\nFortunately, I am committed to keeping up with the coursework. Next week's lab session should feature an assignment due on the same day as our scheduled presentation; you are welcome to direct me there for more information regarding submission details if needed.\n\n\n\nBest regards,\nNaman Gupta"